# Short Enwiki Train

Test that the model init code, runs without issues

**L6-D512 model with**
- Layer count: 6
- Embed size: 512

## Preparing the init model and test dataset

In [2]:
GPU_DEVICES="auto"
ENABLE_WANDB=False
WANDB_PREFIX="infctx-v5-unit-test"
DEEPSPEED_STRAT="deepspeed_stage_1"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: False
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-unit-test
TRAINER_DIR: /home/ubuntu/rwkv-proj/RWKV-infctx-trainer/RWKV-v5
PROJECT_DIR: /home/ubuntu/rwkv-proj/RWKV-infctx-trainer


In [2]:
# First lets setup the various directories
!mkdir -p "{PROJECT_DIR}/model/"
!mkdir -p "{PROJECT_DIR}/datapath/"
!mkdir -p "{PROJECT_DIR}/checkpoint/"

In [3]:
# Lets initialized the L6-D512 model with the init_model.py code
!cd "{TRAINER_DIR}" && python3 init_model.py \
    --n_layer 6 --n_embd 512 \
    --vocab_size world \
    --skip-if-exists --safe-init \
    ../model/L6-D512-world-init.pth

[2023-11-15 04:13:08,468] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0'
---- Initializing model ----
No of layers: 6
Embedding size: 512
Output model path: ../model/L6-D512-world-init.pth
Vocab size: 65536
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
Model exists, skipping init_model


In [4]:
# Preload the dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_10k-world-4x1024.yaml"

/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
Saving the dataset (1/1 shards): 100%|█| 751/751 [00:00<00:00, 11073.50 examples
Saving the dataset (1/1 shards): 100%|███| 8/8 [00:00<00:00, 2278.89 examples/s]


In [7]:
# Short training process - for quick testing / debugging
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_10k-world-4x1024.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (train-ctx=1024, data-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.fast_dev_run=2 \
        --model.load_model="../model/L6-D512-world-init.pth"

[2023-11-15 20:39:11,866] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0'
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-unit-test/config/enwiki_10k-world-4x1024.yaml', '--trainer.logger.init_args.name=infctx-v5-unit-test (train-ctx=1024, data-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.max_steps=2', '--model.load_model=../model/L6-D512-world-init.pth'], args=['fit', '-c', '/home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-unit-test/config/enwiki_10k-world-4

In [ ]:
# Empty out the checkpoint
!cd "{PROJECT_DIR}" && rm -rf "./checkpoint/infctx-v5-unit-test-baseline-4x1024/"

In [5]:
# Longer training process
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_10k-world-4x1024.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (train-ctx=1024, data-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --model.load_model="../model/L6-D512-world-init.pth"
        

[2023-11-15 06:57:49,830] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0'
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-unit-test/config/enwiki_10k-world-4x1024.yaml', '--trainer.logger.init_args.name=infctx-v5-unit-test (train-ctx=1024, data-ctx=4096[][-=], deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--model.load_model=../model/L6-D512-world-init.pth'], args=['fit', '-c', '/home/ubuntu/rwkv-proj/RWKV-infctx-trainer/notebook/trainer-v5-unit-test/config/enwiki_10k-world-4x1024.yaml', '--tra

In [6]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/infctx-v5-unit-test-baseline-4x1024/last.ckpt" "../model/infctx-v5-unit-test-baseline-4x1024.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/infctx-v5-unit-test-baseline-4x1024.pth"

[2023-11-15 07:01:27,247] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/infctx-v5-unit-test-baseline-4x1024/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 1
Parsing checkpoint created by deepspeed==0.12.0
Reconstructed fp32 state dict with 138 params 87601152 elements
Saving bf16 state dict to ../model/infctx-v5-unit-test-baseline-4x1024.pth
-rw-rw-r-- 1 ubuntu ubuntu 168M Nov 15 07:01 ../model/infctx-v5-unit-test-baseline-4x1024.pth


In [8]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && \
    python3 dragon_test.py "../model/infctx-v5-unit-test-baseline-4x1024.pth" "cuda fp32"

[2023-11-15 07:01:43,514] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0'
[SimpleRWKV] Warning: dtype mismatch, only fp32 is supported (for now)
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
[WARNING]: bptt_truncated_learning is set as true (was configured as false), due to incomplete implementation of CUDA kernel for bptt_learning
---
[RWKV.TimeMix] Compiling CUDA kernel with HEAD_SIZE=64
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ld